<a href="https://colab.research.google.com/github/Zfeng0207/FIT3199-FYP/blob/dev%2Fzfeng/model_inferencing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
Mounted at /content/drive


In [2]:
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks (1)/ECG-MIMIC-main')

In [3]:
import os
print(os.getcwd())

/content/drive/.shortcut-targets-by-id/1rFfT7F_iTUUjh7EWzWgquzPn3Fzd6KTD/Colab Notebooks/ECG-MIMIC-main


In [4]:
import os
print(os.listdir())

['LICENSE', 'README.md', 'MIMIC-IV-ECG zip file', 'src', 'reports', 'myenv', 'pretrained_ecg_encoder.pth', 'mimic-iv-2.2', 'saved_models', 'augmentation_examples.png', 'debug_report.txt', 'ecg_model_results', 'ecg_stroke_results', 'ecg_stroke_ensemble', 'enhanced_ecg_results', 'mlruns', 'lightning_logs', 'checkpoints', 'patient_1_12lead_ecg.png', 'patient_3_12lead_ecg.png', 'patient_2_12lead_ecg.png', '__pycache__', 'model.py', 'rnn_attention_model.py', 'stroke_predictions.csv']
['LICENSE', 'README.md', 'MIMIC-IV-ECG zip file', 'src', 'reports', 'myenv', 'pretrained_ecg_encoder.pth', 'mimic-iv-2.2', 'saved_models', 'augmentation_examples.png', 'debug_report.txt', 'ecg_model_results', 'ecg_stroke_results', 'ecg_stroke_ensemble', 'enhanced_ecg_results', 'mlruns', 'lightning_logs', 'checkpoints', 'patient_1_12lead_ecg.png', 'patient_3_12lead_ecg.png', 'patient_2_12lead_ecg.png', '__pycache__', 'model.py', 'rnn_attention_model.py', 'stroke_predictions.csv']


In [5]:
# Step 3: Try importing again
!pip install pytorch_lightning

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.1/823.1 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.1/823.1 kB 14.0 MB/s eta 0:00:00
   ━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/363.4 MB 173.7 MB/s eta 0:00:02Downloading nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl (363.4 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 68.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 68.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 82.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 82.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [6]:
import rnn_attention_model
print(dir(rnn_attention_model))

['BinaryAUROC', 'BinaryAccuracy', 'BinaryF1Score', 'CNN', 'ConvNormPool', 'F', 'LSTMClassifier', 'RNN', 'RNNAttentionModel', 'RNNModel', 'Swish', '__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'nn', 'pad_sequence', 'pl', 'safe_collate', 'torch']


In [7]:
from rnn_attention_model import RNNAttentionModel, ConvNormPool, Swish, RNN, CNN

In [14]:
import torch
import numpy as np
import pandas as pd
import joblib # Import joblib

# Assuming RNNAttentionModel and ConvNormPool are still needed
torch.serialization.add_safe_globals([RNNAttentionModel, ConvNormPool])

# Load model using joblib
model = joblib.load("/content/drive/MyDrive/Colab Notebooks (1)/ECG-MIMIC-main/src/model_input/my_ecg_model.pkl")
model.eval()

# Load data
df = pd.read_csv("src/data/label_df.csv")
print(df.columns)
memmap_path = "/content/drive/MyDrive/Colab Notebooks (1)/ECG-MIMIC-main/src/model_input/memmap_head.npy"
memmap_data = np.memmap(memmap_path, dtype=np.float32, mode='r')

results = []

for idx in range(len(df)):
    subject_id = df.loc[idx, 'study_id']
    start = df.loc[idx, 'start']
    length = df.loc[idx, 'length']

    # Skip invalid lengths
    if length <= 0:
        continue

    # Extract signal
    raw_signal = memmap_data[start: start + length * 12]
    if len(raw_signal) != length * 12:
        continue  # Incomplete signal

    # Normalize and reshape
    normalized = (raw_signal - raw_signal.mean()) / (raw_signal.std() + 1e-6)
    signal = normalized.reshape(length, 12)
    if np.isnan(signal).any() or np.isinf(signal).any():
        continue


    # Predict (Modified for multi-label)
    input_tensor = torch.tensor(signal, dtype=torch.float32).unsqueeze(0)
    with torch.no_grad():
        logits = model(input_tensor)
        probs = torch.sigmoid(logits)  # Get probabilities for all classes
        predictions = (probs > 0.5).int()  # Convert to 0/1 predictions based on threshold

    # Store predictions (Modified to store multiple labels)
    results.append((subject_id, predictions.squeeze(0).tolist()))  # Store list of predictions

# Convert to DataFrame (Modified for multi-label columns)
predictions_df = pd.DataFrame(results, columns=['study_id', 'predictions'])

# Save or print
print(predictions_df.head())
predictions_df.to_csv("stroke_predictions.csv", index=False)

Index(['Unnamed: 0', 'filename', 'study_id', 'patient_id', 'ecg_time',
       'label_train', 'all_diag_all', 'label_stroke', 'start', 'length', 'res',
       'stroke_yn'],
      dtype='object')
   study_id                                 predictions
0  49036311  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
1  48339811  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
2  44069449  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
3  44837313  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
4  40539087  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [15]:
import numpy as np

# Path to your original memmap file
original_memmap_path = "src/data/memmap/memmap.npy"

# Path to save the extracted data
output_memmap_path = "/content/drive/MyDrive/Colab Notebooks (1)/ECG-MIMIC-main/src/model_input/memmap_head.npy"

# Load the original memmap file in read-only mode
original_memmap = np.memmap(original_memmap_path, dtype=np.float32, mode='r')

# Extract the first 36000 elements
extracted_data = original_memmap[:360000]

# Create a new memmap file to store the extracted data
head_memmap = np.memmap(output_memmap_path, dtype=np.float32, mode='w+', shape=extracted_data.shape)

# Copy the extracted data to the new memmap file
head_memmap[:] = extracted_data

# Close the memmap files using del
del head_memmap
del original_memmap

print("Extraction and saving completed successfully!")

Extraction and saving completed successfully!


In [ ]:
import torch
import numpy as np
import pandas as pd

torch.serialization.add_safe_globals([RNNAttentionModel, ConvNormPool])
# Load model
model = torch.load("src/data/full_model.pkl", map_location='cpu', weights_only=False)
model.eval()

# Load data
df = pd.read_csv("src/data/label_df.csv")
print(df.columns)
memmap_path = "src/data/memmap/memmap.npy"
memmap_data = np.memmap(memmap_path, dtype=np.float32, mode='r')

results = []

for idx in range(len(df)):
    subject_id = df.loc[idx, 'study_id']
    start = df.loc[idx, 'start']
    length = df.loc[idx, 'length']

    # Skip invalid lengths
    if length <= 0:
        continue

    # Extract signal
    raw_signal = memmap_data[start: start + length * 12]
    if len(raw_signal) != length * 12:
        continue  # Incomplete signal

    # Normalize and reshape
    normalized = (raw_signal - raw_signal.mean()) / (raw_signal.std() + 1e-6)
    signal = normalized.reshape(length, 12)
    if np.isnan(signal).any() or np.isinf(signal).any():
        continue

    # Predict (Modified for multi-label)
    input_tensor = torch.tensor(signal, dtype=torch.float32).unsqueeze(0)
    with torch.no_grad():
        logits = model(input_tensor)
        probs = torch.sigmoid(logits)  # Get probabilities for all classes
        predictions = (probs > 0.5).int()  # Convert to 0/1 predictions based on threshold

    # Store predictions (Modified to store multiple labels)
    results.append((subject_id, predictions.squeeze(0).tolist()))  # Store list of predictions

# Convert to DataFrame (Modified for multi-label columns)
predictions_df = pd.DataFrame(results, columns=['study_id', 'predictions'])

# Save or print
print(predictions_df.head())
# predictions_df.to_csv("stroke_predictions.csv", index=False)
